In [2]:
!pip install bitarray
!pip install reedsolo
import math
import bitarray
import hashlib
from reedsolo import RSCodec

     |████████████████████████████████| 88 kB 5.7 MB/s 
  Created wheel for bitarray: filename=bitarray-2.3.2-cp37-cp37m-linux_x86_64.whl size=171952 sha256=ef299e56d6b865af0063bc5ad6bfbb0a5e1688ba5cdaf45c58ee0a140d082867
  Stored in directory: /root/.cache/pip/wheels/02/3a/55/c4aba43815b20355bd5fc7219faae18315ae9f5bcd5393acf7
Successfully built bitarray
     |████████████████████████████████| 271 kB 27.9 MB/s 
  Created wheel for reedsolo: filename=reedsolo-1.5.4-cp37-cp37m-linux_x86_64.whl size=706444 sha256=99af31483b7b7603c171a82530f903dfcbd81e873099225888d5e7df147c1d20
  Stored in directory: /root/.cache/pip/wheels/20/b9/92/6b9dcd4ed356565c382d97443dfb6206f614c032149402a49d
Successfully built reedsolo


In [68]:
#definitions & values

partschema1 = "20|ivory navy honeydew sandy midnight|Manufacturer#1|Brand#12|LARGE POLISHED NICKEL|48|MED BAG|920.02|are across the asympt|"

partschema = "partkey|name|mfgr|brand|type|size|container|retailprice|comment|"
d = "8|misty lace thistle snow royal|Manufacturer#4|Brand#44|PROMO BURNISHED TIN|41|LG DRUM|908.00|eposi|"
e = "7|moccasin green thistle khak|Manufacturer#1|Brand#11|SMALL PLATED COPPER|45|SM BAG|907.00|lyly. ex|"
d2 = "9|misty lace thistle snow royal|Manufacturer#1|Brand#11|SMALL PLATED COPPER|45|SM BAG|907.00|lyly. ex|"

partsupschema = "partkey|suppkey|availqty|supplycost|comment|"
g = "8|79|9845|220.62|s against the fluffily special packages snooze slyly slyly regular p|"
h = "7|61|2770|149.66|hould have to nag after the blithely final asymptotes. fluffily spe|"

hexlength = 9
hexlengthkey = 1

def encode2bits(d):
    return {
        '00': 'A',
        '01': 'C',
        '10': 'G',
        '11': 'T',
    }[d]

def encodejunk(lastBase, ba):
    lb =lastBase

    if(ba == '1'):
        if(lb == 'A'):
            lb = 'C'
        elif(lb == 'C'):
            lb = 'A'
        else:
            lb = 'A' #random.choice(['A', 'C'])
    else:
        if (lb == 'G'):
            lb = 'T'
        elif(lb == 'T'):
            lb = 'G'
        else:
            lb = 'T' #random.choice(['G', 'T'])

    return lb


def encode(value, ul, jl):
    #length of useful sequences
    #ul = 1
    #value = 102
    res = []

    bvalue = ''

    if (type(value) is str):

        t = bitarray.bitarray()
        t.frombytes(value.encode('utf-8'))

        bi = 0

        for i in range (0, len(value)):
            #cut out first bit
            for j in range (0, 1):
               bi += 1

            for k in range (0, 7):
                if(t[bi]):
                    bvalue += '1'
                else:
                    bvalue += '0'

                bi += 1

            #bvalue += 'X'

    else:
        bvalue = '{0:08b}'.format(value)
        #print(bvalue)

    s = hashlib.shake_256()
    s.update(str(value).encode())
    for i in range(0, int(len(bvalue) / 2)):
        d2 = "%s" % bvalue[i * 2:i * 2 + 2]
        res.append(encode2bits(d2))

    # remove all leading zeros
    #while res[0] == 'A':
    #    res.pop(0)

    result = []
    # desired length of each junk sequence
    #jl = 2
    # jl = int(len(bjunk)/len(res))
    # determining the length of the hexdigest so we have enough junk
    rjl = math.ceil(jl * len(res) / 8) * 8
    bjunk = bin(int(s.hexdigest(int(rjl / 8)), 16))[2:].zfill(rjl)
    #print("Binary hash")
    #print(bjunk)
    # jl = math.floor(len(bjunk)/len(res))
    # counter
    jp = 0
    for i in range(0, int(len(res))):

        for j in range(0, ul):
            result.append(res[i])
            i += 1

        for k in range(0, jl):
            j = encodejunk(result[len(result) - 1], bjunk[jp])
            jp += 1
            result.append(j)

    # append rest of junk sequence so we have the entire hex digest
        #    for i in range(jp, len(bjunk)):
        #        j = encodejunk(result[len(result) - 1], bjunk[jp])
        #        jp += 1
        #        result.append(j)

    #print("".join(result))

    return "".join(result)


def decode(value, ul, jl, resulttype):
    # input sequence
    #value = 'CTAGTCCACGTC'
    # bit length of variable/field
    #noul = 4
    # requested length of junk sequence
    #jl = 2
    # length of useful sequences
    #ul = 1

    def decode1base(d):
        return {
            'A': '00',
            'C': '01',
            'G': '10',
            'T': '11',
        }[d]

    def decodejunk(ba):

        if (ba == 'A' or ba == 'C'):
            return '1'
        else:
            return '0'

    index = 0
    val = []
    checksum = []

    noul = int(len(value)/(ul+jl))

    for i in range(0, noul):

        for j in range(0, ul):
            #bit ugly this

            val.append(decode1base(value[index]))
            index += 1

        for k in range(0, jl):
            checksum.append(decodejunk(value[index]))
            index += 1

        #for i in range(index, len(value)):
        #checksum.append(decodejunk(value[index]))
        #index += 1

    if(resulttype == "str"):
        vs = "".join(val)

        val2 = []

        for i in range (0, len(vs)):

            if(i%7 == 0):
                val2.append('0')

            val2.append(vs[i])


        result = bitarray.bitarray("".join(val2)).tobytes().decode('utf-8')

    elif (resulttype == "int"):
        result = int("".join(val), 2)
    else:
        print("Undefined return type")


    #print(result)
    #print(hex(int("".join(checksum), 2)))

    #resulting from 5bit character encoding
    if (resulttype == "str"):
        result = result.replace('`', ' ')

    s = hashlib.shake_256()
    s.update(str(result).encode())

    #length of the digest we care for

    dl = noul * jl
    rjl = math.ceil(jl * dl / 8) * 8
    bdigest = bin(int(s.hexdigest(int(rjl / 8)), 16))[2:].zfill(rjl)

    #print(hex(int(bdigest[0:dl], 2)) + " == " + hex(int("".join(checksum), 2)))

    return result


def decompose(record, table, schema, pk):
    ratoms = record.split("|")
    satoms = schema.split("|")

    pkey = ratoms[pk]

    for r, s in zip(ratoms, satoms):
        if s and r:

            h = hashlib.shake_256()
            h.update(str(r).encode())
            ha = h.hexdigest(hexlength)

            h.update(str(pkey).encode())
            hp = h.hexdigest(hexlengthkey)

            ea = encode(ha, 1, 1)
            ep = encode(hp, 1, 1)

            ihok = int(hp, 16)
            rs = RSCodec(4)
            b_ihok = bytes(str(ihok), encoding='utf8')
            teccpk = rs.encode(b_ihok)
            eccpk = teccpk[len(str(ihok)):]

            eeccpk = ''

            for by in eccpk:
                eeccpk = eeccpk + encode(by, 1, 1)

            print(table + " " + s + " " + pkey + " " + r)
            print(table + " " + s + " " + hp + " " + ha)
            print(table + " " + s + " " + eeccpk + " " + ep + " " + ea)

            n = int(len(eeccpk)/4)
            els = [eeccpk[i:i + n] for i in range(0, len(eeccpk), n)]

            tba = bytearray(str(ihok-1), "latin-1")

            for e in els:
                val = decode(e, 1, 1, 'int')
                tba.append(val)

            r = rs.decode(tba)
            print(r)

decompose(partschema1, "part", partschema, 0)

part partkey 20 20
part partkey 29 56ed42d09d2b99b6d7
part partkey ATCTATCAGTCACACTGTATATCTATACACAC CAGTCTACTAGACA CAGAGAGTTACAGATAATGATAGTCAATCAGAGTACTGATGATGATGTATTAACATCATGATTGGACAATCTGACACAGAATGACATAATGATAGACATGACCTATTGCAGTTGACGTACTACTTA
(bytearray(b'41'), bytearray(b'41\x11\x95\x81\x00'), bytearray(b'\x01'))
part name 20 ivory navy honeydew sandy midnight
part name ec 3a69cae50a5ed1183a
part name CTCTCAGTATTAACCTTACTATGTGTATACGA TAACGATGGAATTA CTGTCTTAGAATCACTGATGATTGGTCATAATCTTGGTATCATAACGTGTTGCTCACTGAATCTGTATCTCAGAGATGGACACTTAATGTACTAACCTCTGTATGTTAGAATCAGTCTTAGTACCT
(bytearray(b'236'), bytearray(b'236V1\xd2\x82'), bytearray(b'\x02'))
part mfgr 20 Manufacturer#1
part mfgr 57 fce78907aa806a8ebb
part mfgr GACTTACTTGGAATGTATTATGCACAATTACA CTGAGTGTTGCTTA TAACTACTGTACTGTGATGAGTTGCATACATGACACTAGTCACTGAATATTGCTTATGATATTAGTACCTCTTGACACTGATATCTGATGCAGTACCACTTGACCAGACACTTGACCACTGTACGA
(bytearray(b'87'), bytearray(b'87\x9d\xe2=M'), bytearray(b'\x01'))
part brand 20 Brand#12
part brand b7 d6